<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_5_Harmonization_%26_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The goal of this module is to take the flood maps that we obtained from the global products, mosaic them, clip them, and assign to them all common raster values.

Each of the products has their own unique raster values, which can be seen in [this table](https://docs.google.com/spreadsheets/d/15igmiH1EHjtiyprgx6LnF78lvNh2scZSnlc12ySr0PQ/edit?usp=sharing). This script will take each product and reassign its raster values to the following raster value scheme:

Pixel Value            |        Land Cover Class
---                    | :---:
0                      |  Nonwater
1                      |  Water
2                      |  Masked (e.g. cloud, cloud shadow)

**Table 1: Pixel Values Used for SERVIR's Flood Mapping Intercomparison Project**

In [ ]:
#!pip install geemap

In [ ]:
import geemap
import ee

ee.Authenticate()
ee.Initialize(project='servir-sco-assets')

# Step 1: Inputs

Replace the variables below with the parent directory that you uploaded your images to as shown in Module 5.

Define your parent directory below.

**MODIFIABLE VARIABLE ALERT**

In [ ]:
# Define the parent directory where the flood products we obtained in module 5 are located
parent_directory = "projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/"

n_mcdwd_images = 12      # the number of mcdwd images in the parent directory
n_dswx_images = 44       # the number of dswx images in the parent directory
n_vfm_images = 2         # the number of vfm images in the parent directory
n_gfm_images = 12        # the number of gfm images in the parent directory
n_hydrosar_images = 6    # the number of hydrosar images in the parent directory

# Step 2: Compositing

## Define AOI to clip Composites to

In [ ]:
aoi = ee.FeatureCollection('users/mickymags/flood_intercomparison/aoi')

# Get the coordinates of the center of the AOI for mapping purposes
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

In [ ]:
lon

73.87256778127869

In [ ]:
lat

31.027857639571568

## MCDWD Compositing

In [ ]:
mcdwd_ic = ee.ImageCollection([]) # Create an empty Image Collection

for i in range(n_mcdwd_images):
  instring = parent_directory + 'mcdwd/' + 'mcdwd_pt' + str(i + 1)
  asset = ee.Image(instring)
  asset2 = ee.ImageCollection(asset)
  mcdwd_ic = mcdwd_ic.merge(asset2)

mcdwd_mos = mcdwd_ic.mosaic().clip(aoi)

## VFM Compositing

In [ ]:
vfm_ic = ee.ImageCollection([]) # Create an empty Image Collection

for k in range(n_vfm_images):
  instring = parent_directory + 'vfm/' + 'vfm_pt' + str(k + 1)
  asset = ee.Image(instring)
  asset2 = ee.ImageCollection(asset)
  vfm_ic = vfm_ic.merge(asset2)

vfm_mos = vfm_ic.mosaic().clip(aoi)

## GFM Compositing

In [ ]:
gfm_ic = ee.ImageCollection([])

for m in range(n_gfm_images):
  instring = parent_directory + 'gfmpt2/' + 'gfmpt' + str(m+1)
  print(instring)
  asset = ee.Image(instring)
  asset2 = ee.ImageCollection(asset)
  gfm_ic = gfm_ic.merge(asset2)

gfm_mos = gfm_ic.mosaic().clip(aoi)

projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt1
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt2
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt3
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt4
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt5
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt6
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt7
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt8
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt9
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt10
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmpt2/gfmpt11
projects/servir-sco-assets/assets/flood_intercomparison/pk_case_study/gfmp

## DSWx-HLS compositng

In [ ]:
dswx_ic = ee.ImageCollection([])

for j in range(n_dswx_images):
  instring = parent_directory + 'dswx/' + 'dswx_pt' + str(j+1)
  asset = ee.Image(instring)
  asset2 = ee.ImageCollection(asset)
  dswx_ic = dswx_ic.merge(asset2)

dswx_mos = dswx_ic.mosaic().clip(aoi)

# HYDROSAR

In [ ]:
hydrosar_ic = ee.ImageCollection([])

for u in range(n_hydrosar_images):
  instring = parent_directory + 'hydrosar/' + 'hydrosar_pt' + str(u+1)
  asset = ee.Image(instring)
  asset2 = ee.ImageCollection(asset)
  hydrosar_ic = hydrosar_ic.merge(asset2)

hydrosar_final = hydrosar_ic.mosaic().clip(aoi)

# HYDRAFloods

In [ ]:
hf = ee.Image("users/mickymags/fmi/hf_pk_06162023")
hf_final = hf.clip(aoi)

# Visualization

## HYDROSAR compositing

In [ ]:
'''
aoi = ee.FeatureCollection("projects/servir-sco-assets/assets/flood_intercomparison/chad_cs/chad_case_study")

# VIIRS
vfm1 =  ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/vfm/take2_scene95")   # Scene 1
vfm2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/vfm/take2_scene96")    # Scene 2

# GFM
gfm1 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_1_scene_1')
gfm2 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_1_scene_2')
gfm3 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_2_scene1')
gfm4 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath2_scene2')
gfm5 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath2_scene3')
gfm6 = ee.Image('projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath2_scene4')

# MCDWD
# Under Construction -- Repeat for a case study where we have MCDWD data

# OPERA DSWx
# Under Construction -- Repeat for a case study where we have OPERA DSWx.

# HYDRAFloods
# Under Construction -- Import HYDRAFloods for visualization purposes later

# HYDROSAR
# Under Construction -- Import HYDROSAR for visualization purposes later
'''

'\naoi = ee.FeatureCollection("projects/servir-sco-assets/assets/flood_intercomparison/chad_cs/chad_case_study")\n\n# VIIRS\nvfm1 =  ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/vfm/take2_scene95")   # Scene 1\nvfm2 = ee.Image("projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/vfm/take2_scene96")    # Scene 2\n\n# GFM\ngfm1 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_1_scene_1\')\ngfm2 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_1_scene_2\')\ngfm3 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath_2_scene1\')\ngfm4 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath2_scene2\')\ngfm5 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercomparison/chad_nov_2020/gfm/swath2_scene3\')\ngfm6 = ee.Image(\'projects/servir-sco-assets/assets/flood_intercompar

# Part 3: Harmonize

## 3.1 VFM Harmonization

In [ ]:
#############################################################################
#   Part 2: Create a function to reassign pixel values to our new schema    #
#############################################################################
# Define a function to transform VFM maps into common schema. This function takes an image as
#its input and outputs an image with the schema mentioned in Table 1
def vng_sch(image):

  ################ Get Water Pixels ####################

  # Raster Values of 15 indicate open water without water fraction retrieval
  # Raster Values of 99 indicate
  # Raster Values of 100 indicate Open Normal Water, from a river, lake, reservoir, or ocean
  # Raster Values of 101-200 idnicate water fractions of floodwater. We selected 50 as the percent threshold
  # to consider a pixel as water/nonwater. Thus, for the purposes of this study, we selected 150 as the threshold
  # above which pixels are considered as floodwater pixels.
  vfm_water = image.eq(15).Or(image.eq(99)).Or(image.eq(100)).Or(image.gte(150))

  ############## Get Nonwater Pixels ########################

  # Part 1 will get pixels that have floodwater but have less than 50% of the pixel covered in floodwater
  # Raster Values of 16 indicate clear-sky bare land
  # Raster Values of 17 indicate clear-sky vegetation
  # Raster Values of 20 indicate snow cover
  # Raster Values of 27 indicate river/lake covered in ice
  # Raster values of 38 indicate supra-snow ice water, mixed ice and water, or ice in melting status
  pt1 = image.gt(100).And(image.lt(150))
  pt2 = image.eq(16).Or(image.eq(17)).Or(image.eq(20)).Or(image.eq(27)).Or(image.eq(38))
  vfm_nonwater = pt1.Or(pt2)

  ############## Get Pixels where we do not have clear observations #############

  # Values of 1 indicate bad data pixels
  # Values of 30 indicate pixels where there is cloud cover
  # Values of 50 indicate pixels that have shadows from clouds or terrain.
  vfm_mask = image.eq(1).Or(image.eq(30)).Or(image.eq(50))

  # This section of the
  #ones = ee.Image(1)   # Dummy Image where every pixel has a value of 1
  #zeros = ee.Image(0)  # Dummy Image where every pixel has a value of 0
  #twos = ee.Image(2)   # Dummy Image where every pixel has a value of 2

  # This section of the code will replace the VFM raster values with our classification schema (see Table 1).
  vfm_mod = image.where(vfm_nonwater, ee.Image(0))   # Replace Nonwater Pixels as found by this code with a value of 0   # zeros
  vfm_v2 = vfm_mod.where(vfm_water, ee.Image(1))     # Replace Water Pixels as found by this code with a value of 1    # ones
  vfm_v3 = vfm_v2.where(vfm_mask, ee.Image(2))       # Replace Mask Pixels as found by this code with a value of 2    # twos

  return ee.Image(vfm_v3)

#################################################
#   Part 3: Apply our function from Part 2      #
#################################################
# Apply the dswx_sch function to our clipped dswx mosaic to the schema
# shown in table 1.
# Run the function on the mosaic
vfm_final = ee.Image(vng_sch(vfm_mos))

# Section 2: GFM

We do not need to transform the GFM map into our common schema as it already has that schema. Thus, all we will do is combine all of the GFM scenes into a mosaic and clip that mosaic to the area of interest.

In [ ]:
# We want to assign the pixels that were masked in the GFM image to have a value of 2.
def gfm_sch(image, aroi):
  img_2 = ee.Image(2).clip(aroi)
  wawa = image.eq(1)
  nowawa = image.eq(0)

  ones = ee.Image(1)
  zeros = ee.Image(0)

  gfm_new = img_2.where(wawa, ones)
  gfm_fin = gfm_new.where(nowawa, zeros)
  return gfm_fin

# Mosaic the individual GFM images and clip the mosaic to the area of interest
gfm_final = gfm_sch(gfm_mos, aoi)

In [ ]:
vp = {
    'min': 0,
    'max': 2,
    'palette': ['000000', 'FF0000', 'FFFFFF']
}

In [ ]:
Map = geemap.Map(center = (lat, lon), zoom = 5)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(gfm_final, vp, 'GFM Map')
Map.addLayer(gfm_mos, vp, 'GFM Mos')
#Map.addLayer(gfm_mask, {}, 'GFM')
#Map.addLayer(hydrafloods, vp, 'VIIRS Flood Map')
#Map.addLayer(hydrosar, vp, 'VIIRS Flood Map')

Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

# Section 3: MCDWD


In [ ]:
#####################################################
#  Part 1: Mosaic MCDWD Images, Clip them to aoi    #
#####################################################
#mcdwd_mos = ee.ImageCollection([mcdwd1, mcdwd2, mcdwd3]).mosaic().clip(aoi)

#############################################################################
#   Part 2: Create a function to reassign pixel values to our new schema    #
#############################################################################
def mcdwd_sch(image):

  #image = image.unmask()

  ################ Get Water Pixels ####################

  # Pixel Values of 1 indicate regular surface water according to the MCDWD algorithm
  # Pixel Values of 2 indicate a recurring flood according to the MCDWD algorithm
  # Pixel values of 3 indicate floodwater according to the MCDWD algorithm
  mcdwd_water = image.eq(1).Or(image.eq(2)).Or(image.eq(3))

  ############## Get Nonwater Values ########################
  # Pixel values of 0 indicate no water is present according to the MCDWD algorithm
  mcdwd_nonwater = image.eq(0)

  ############## Get Pixels where we do not have clear observations #############
  # Pixel values of 255 indicate insufficient data
  #mcdwd_mask = image.mask()               # Get the mask of the MCDWD map
  #mcdwd_maskdata = mcdwd_mask.eq(0)       # Find values where the MCDWD mask is equal to 0
  #mcdwd_insufficient = image.eq(255)            # Get the MCDWD values with insufficient data
  #mcdwd_unclear = mcdwd_maskdata.Or(mcdwd_insufficient)

  # Replace MCDWD pixel values with our classification schema (see Table 1)
  #mcdwd_mod = image.where(mcdwd_nonwater, ee.Image(0))
  #mcdwd_v2 = mcdwd_mod.where(mcdwd_water, ee.Image(1))
  #mcdwd_v3 = mcdwd_v2.where(mcdwd_unclear, ee.Image(2))
  #mcdwd_v3 = image.where(mcdwd_maskdata, ee.Image(2))

  mcdwd_mask = image.mask().clip(aoi)
  mcdwd_maskdata = mcdwd_mask.eq(0)
  mcdwd_insufficient = image.unmask().eq(255)
  mcdwd_unclear = mcdwd_maskdata.Or(mcdwd_insufficient)

  #mcdwd_v3 = image.unmask().where(mcdwd_maskdata, ee.Image(2))
  mcdwd_v0 = image.unmask()
  mcdwd_v1 = mcdwd_v0.where(mcdwd_nonwater, ee.Image(0))
  mcdwd_v2 = mcdwd_v1.where(mcdwd_water, ee.Image(1))
  mcdwd_v3 = mcdwd_v2.where(mcdwd_unclear, ee.Image(2))
  mcdwd_v4 = mcdwd_v3.clip(aoi)

  #return mcdwd_v3#.clip(aoi)
  return mcdwd_v4

#################################################
#   Part 3: Apply our function from Part 2      #
#################################################
# Apply the mcdwd_sch function to our clipped dswx mosaic to the schema
# shown in table 1.
mcdwd_final = mcdwd_sch(mcdwd_mos)

# Section 4: OPERA DSWx

In [ ]:

#####################################################
#   Part 1: Mosaic DSWx Images, Clip them to aoi    #
#####################################################
#dswx_mos = ee.ImageCollection([dswx1, dswx2]).mosaic().clip(aoi)

##############################################################################
#   Part 2: Create a function to Reassign pixel values to our new schema     #
##############################################################################
def dswx_sch(image):

  ################ Get Water Pixels ####################
  # Find Areas in DSWx that are water
  # Pixel Values of 1 are water.
  # Pixel Values of 254 are ocean masked
  dswx_water = image.eq(1).Or(image.eq(254))

  ############### Get Nonwater Pixels ########################

  # Pixel values of 0 are nonwater according to dswx
  # Pixel values of 252 are snow or ice according to dswx
  dswx_nonwater = image.eq(0).Or(image.eq(252))

  ############## Get Pixels where we do not have clear observations #############
  # Pixel Values of 255 are Fill Values according to dswx
  # Pixel Values of 253 are associated with clouds or cloud shadows
  dswx_mask = image.eq(255).Or(image.eq(253))

  # Assign pixel values of 0 to areas classified as nonwater by dswx,
  # pixel values of 1 to areas classified as water by dswx, and pixel
  # values of 2 to areas classified as neither water nor nonwater by dswx.
  dswx_mod = image.where(dswx_nonwater, ee.Image(0))
  dswx_v2 = dswx_mod.where(dswx_water, ee.Image(1))
  dswx_v3 = dswx_v2.where(dswx_mask, ee.Image(2))

  # The output of this function will be the reclassified image.
  return dswx_v3

#################################################
#   Part 3: Apply our function from Part 2      #
#################################################
# Apply the dswx_sch function to our clipped dswx mosaic to the schema
# shown in table 1.
dswx_final = dswx_sch(dswx_mos)

# Section 5: Visualize the Flood Maps

Let's Visualize the Flood Maps to get a general idea of how they perform when compared with each other.

In [ ]:
Mmaskvp = {
    'min': 0,
    'max': 1,
    'palette': ['000000', 'FFFFFF']
}

In [ ]:
mcdwd_mos_vp = {
    'min': 0,
    'max': 4,
    'palette': ['ff0000', 'ffa500', 'ffff00', '00ff00', '0000ff']
}

In [ ]:
vp = {
    'min': 0,
    'max': 2,
    'palette': ['000000', 'add8e6', 'FFFFFF']
}

# Enter the latitude and latitude of the center of your aoi

Map = geemap.Map(center = (lat, lon), zoom = 5)
Map.addLayer(aoi, {}, 'Area of Interest')
Map.addLayer(vfm_final, vp, 'VIIRS Flood Map')
Map.addLayer(mcdwd_final, vp, 'MCDWD Flood Map')
Map.addLayer(mcdwd_mos, mcdwd_mos_vp, 'MCDWD Mosaic')
#Map.addLayer(Mmcdwdunmask, {}, 'MCDWD Mosaic Unmasked')
#Map.addLayer(Mmcdwdmasktest, Mmaskvp, 'Mask test')
Map.addLayer(gfm_final, vp, 'GFM Map')
Map.addLayer(dswx_final, vp, 'DSWX Flood Map')
#Map.addLayer(Mcloudy, Mmaskvp, 'Cloudy')
Map.addLayer(hf_final, vp, 'HYDRAFloods ')
Map.addLayer(hydrosar_final, vp, 'HYDROSAR')

Map.addLayerControl()
Map

Map(center=[31.027857639571568, 73.87256778127869], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
#Mmcdwdunmask = mcdwd_mos.unmask().clip(aoi)

In [ ]:
#Mmcdwdmasktest = mcdwd_mos.mask().clip(aoi)
#Mcloudy = Mmcdwdmasktest.eq(0)

In [ ]:
'''
vp = {
    'min': 0,
    'max': 2,
    'palette': ['000000', 'add8e6', 'FFFFFF']
}

# Enter the latitude and latitude of the center of your aoi

Map = geemap.Map(center = (lat, lon), zoom = 5)
Map.addLayer(aoi, {}, 'Area of Interest')
#Map.addLayer(vfm_final, vp, 'VIIRS Flood Map')
Map.addLayer(mcdwd_final, vp, 'VIIRS Flood Map')
#Map.addLayer(gfm_final, vp, 'GFM Map')
#Map.addLayer(dswx_final, vp, 'VIIRS Flood Map')
#Map.addLayer(hydrafloods, vp, 'VIIRS Flood Map')
#Map.addLayer(hydrosar, vp, 'VIIRS Flood Map')

Map.addLayerControl()
Map
'''

"\nvp = {\n    'min': 0,\n    'max': 2,\n    'palette': ['000000', 'add8e6', 'FFFFFF']\n}\n\n# Enter the latitude and latitude of the center of your aoi\n\nMap = geemap.Map(center = (lat, lon), zoom = 5)\nMap.addLayer(aoi, {}, 'Area of Interest')\n#Map.addLayer(vfm_final, vp, 'VIIRS Flood Map')\nMap.addLayer(mcdwd_final, vp, 'VIIRS Flood Map')\n#Map.addLayer(gfm_final, vp, 'GFM Map')\n#Map.addLayer(dswx_final, vp, 'VIIRS Flood Map')\n#Map.addLayer(hydrafloods, vp, 'VIIRS Flood Map')\n#Map.addLayer(hydrosar, vp, 'VIIRS Flood Map')\n\nMap.addLayerControl()\nMap\n"

# Section 6: Export the Mosaicked Flood Maps with new Raster Values

Now we will Export our new images to Google Earth Engine images so we can use them later.

In [ ]:
# Define a function that exports an Image to a Google Earth Engine Asset.
def exporter(img, asset_id, scale_):

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = 'EPSG:4326',
                                  scale = scale_,
                                  maxPixels = 1e13)
  return 0

In [ ]:
# Export VFM
vfm_aid_final = parent_directory + 'vfm/vfm_mosaic'
#exporter(vfm_final, vfm_aid_final, 375)

# Export GFM
gfm_aid_final = parent_directory + 'gfm/gfm_mosaic'
#exporter(gfm_final, gfm_aid_final, 30)

# Export MCDWD
mcdwd_aid_final = parent_directory + 'mcdwd/mcdwd_mosaic'
#exporter(mcdwd_final, mcdwd_aid_final, 250)

# Export OPERA DSWx
dswx_aid_final = parent_directory + 'dswx/dswx_mosaic'
#exporter(dswx_final, dswx_aid_final, 20)""

# Export Hydrosar
hydrosar_aid_final = parent_directory + 'hydrosar/hydrosar_mosaic'
exporter(hydrosar_final, hydrosar_aid_final, 30)

# Export HYDRAFloods
hydrafloods_aid_final = parent_directory + 'hydrafloods/hydrafloods_mosaic'
#exporter(hf_final, hydrafloods_aid_final, 30)

The above cell will return a 0, but your exports have begun. Click [here](https://code.earthengine.google.com/) to open the GEE Code Editor Interface. Navigate to the Tasks tab by clicking on the right tab. These will take a couple minutes to an hour to run depending on the size of your study area.